# Using the OpenAI API from the JupyterLite JS Kernel

> join the discussion on [Jupyter Discourse](https://discourse.jupyter.org/t/how-to-install-openai-npm-package-in-the-javascript-kernel-in-jupyterlite/19065)

## You need a key

If you don't have one, this isn't going to work at all.

In [ ]:
OPEN_API_KEY = "GET YOUR OWN KEY";

## Get Modules

There's a lot of code needed, and `async` things don't work quite right inside the JS Kernel WebWorker yet. [jspm.io](https://jspm.org/) provides a CDN with shims. 

In [ ]:
CDN = "https://ga.jspm.io/npm"

to get to something more like the documentation for [`importMap`](https://developer.mozilla.org/en-US/docs/Web/HTML/Element/script/type/importmap)s.

In [ ]:
importScripts(`${CDN}:es-module-shims@1.6.2/dist/es-module-shims.wasm.js`);

## Set Import Map

The upstream code on `npm.js` isn't directly usable, and needs to have first been transpiled to [ES Modules](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Guide/Modules).

In [ ]:
importShim.addImportMap({
    "imports": {
        "openai": `${CDN}:openai@3.2.1/dist/index.js`
    },
    "scopes": {
      "https://ga.jspm.io/": {
        "#lib/adapters/http.js": `${CDN}:axios@0.26.1/lib/adapters/xhr.js`,
        "axios": `${CDN}:axios@0.26.1/index.js`,
        "form-data": `${CDN}:form-data@4.0.0/lib/browser.js`
      }
    }
  });

## Main

Again, as the worker doesn't support top-level `await`, you'll need a function to do any async stuff (which is pretty much everything), with the following lightly adapted from the [documentation](https://github.com/openai/openai-node#usage).

In [ ]:
async function main () {
    const { Configuration, OpenAIApi } = await importShim('openai');
    
    const configuration = new Configuration({
        apiKey: OPEN_API_KEY,
    });

    const openai = new OpenAIApi(configuration);

    const completion = await openai.createCompletion({
        model: "text-davinci-003",
        prompt: "Hello world",
    });
    
    console.log(completion.data.choices[0].text);
}

async function mainWithCatch(prompt){
    try {
        return await main()
    } catch(err) {
        console.error(err)
    }
}

// actually call main
void mainWithCatch();